# Design of a Tension Member and End Connections

| Property | Value |
|----------|:-------:|
|  $T_f$   | 800 kN|
| Angles   | 300W  |
| Plates   | 300W |
| HSS      | 350W |
| W        | A992 |
| Bolts    | A325 3/4" |
| Holes    | 22 mm punched |
| Electrodes | E49xx |

In [1]:
# set the default precion for dispaly of floating point numbers
%precision 3

'%.3f'

In [2]:
phi = 0.9
phiu = 0.75
phib = 0.8
phibr = 0.8
phiw = 0.67

aFy = 300   # angles
aFu = 440
pFy = 300   # plates
pFu = 440
hFy = 350   # HSS
hFu = 450
wFy = 345   # W shapes
wFu = 450
bFu = 825   # bolt
Xu = 490    # weld
bd = 25.4 * 0.75   # 3/4" bolt
hd = 22 + 2       # in 22mm punched holes

In [3]:
Tf = 800

In [4]:
from sst import SST
sst = SST()

Tension brace will be bolted into frame.  Net area fracture is often the governing case.  Reasonable
estimates are $A_{n} = 0.9 A_g$ and $A_ne = 0.85 A_n$.  Therefore, a first estimate of cross-sectional
area required is, without differentiating material strengths, is:

$$
\phi_u \times 0.85 \times 0.9 A_g \times Fu \ge T_f
$$

$$
A_g \ge \frac{T_f}{\phi_u 0.85 \times 0.9 Fu}
$$

In [5]:
Tf/(phi*0.85*0.9*450*1E-3)

2582.103

Therefore, look for a square HSS of at least this cross-sectional area:

>   _Gentle Reader:_ If you find the following 'python-pandas magic' hard to understand,
>   don't worry about it.  Its just the digital equivalent of searching through
>   the properties and dimensions tables in Part 6 of the CISC handbook of Steel Construction.

In [6]:
# load a table of all HSS sections, sorted by increasing mass
hss = sst.section_tables(['HS'],'Mass')

# search for those shapes whose area meets the limit and whose B and D dimensions are equal (square)
# Then extract the first 10 rows (shapes) and choose only three columns for display.
hss.query('A>2500 & B==D').head(10)[['A','T','Mass']]

,A,T,Mass
Dsg,,,
HS114x114x6.4,2640.0,6.35,20.7
HS152x152x4.8,2760.0,4.78,21.7
HS89x89x9.5,2790.0,9.53,21.9
HS102x102x8.0,2820.0,7.95,22.1
HS127x127x6.4,2960.0,6.35,23.2
HS114x114x8.0,3220.0,7.95,25.3
HS178x178x4.8,3250.0,4.78,25.5
HS102x102x9.5,3280.0,9.53,25.7
HS152x152x6.4,3610.0,6.35,28.3


Probably do not want the thinnest section (because of welding considerations). 
Also, a larger dimension may be easier to connect.
And leave a bit of allowance 
over the minimum.  Therefore:

### Try: HSS 127x127x6.4

In a configuration similar to this:

![Photo](170-DSCN0768.JPG)

In [7]:
##angle=sst.section_tables(['L'],'Mass')
##angle.query("Avl!='*'&A>2500/4")[['A','T','Mass']].head(10)

In [8]:
##angle=sst.section_tables(['W'],'Mass')
##angle.query("Avl!='*'&A>2500")[['Dsg','A','T','Mass']].head(10)

#### Bolting Requirements

Again, because net area considerations usually govern we should start by selecting a bolt configuration (number of bolts and layout).  That will allow us to decide gusset plate thickness, which is important because the insert
plate welded into the HSS must be the same thickness, and that affects the strength of the HSS.

In [9]:
Ab = 3.14159*bd*bd/4   # bolt shear area
vr1 = 0.60*phib*1*2*Ab*bFu * 0.7 * 1E-3   # factored shear strength of 1 bolt, double shear, threads intercepted
Tf/vr1                 # number of bolts required

5.063

Therefore, use 6 bolts.  But should they be in a 2x3 or a 3x2 arrangement?

![Layout](bolt-layout.svg)

### Try 2x3 bolt pattern
2 gauge lines, 3 transverse lines.  Use Gauge = 75mm, spacing = 75mm (minimum = 2.7d = 52mm).

From Table 6, minimum edge/end distance = 32mm (assuming sheared edges). Use edge and end distances of 35mm.

#### Gusset plate design
Mostly we want the thickness so that we can determine the thickness of the insert plate in the HSS.

As the gusset plate can be made as wide as necessary, design will likely be governed by block shear and/or bearing.

Calculate resistances of a 1mm thickness of plate, then determine the thickness required.

![Gusset Plate](gusset-plate.svg)

![Gusset Failure Paths](gusset-failure-paths.svg)

##### Path 1-2 (Block Shear) (1mm plate)

In [10]:
An = (75 - (hd/2)*2)*1
Ut = 1
An

51.000

In [11]:
Agv = (2*(2*75+35))*1
Agv

370

In [12]:
vr1 = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
vr1

78.435

##### Path 1-3 (Block Shear)  (1mm plate)

In [13]:
An = (75+35-(hd/2)*3)*1
Ut = 0.6
An

74.000

In [14]:
Agv = (1*(2*75+35))*1
Agv

185

In [15]:
vr2 = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
vr2

45.455

##### Path 4-4 (Tearout)  (1mm plate)

In [16]:
An = 0.
Ut = 1

In [17]:
Agv = (4*(2*75+35))*1
Agv

740

In [18]:
vr3 = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
vr3

123.210

##### Path 5-3 (Net Section Fracture)  (1mm plate)

![Failure Path](gusset-net-section.svg)

In [19]:
# length 7-3:
l73 = 75 + 35
# length 6-7: (all lines at 45 degrees)
l67 = (75+75)/(2**0.5)
# length 5-6:
l56 = 35*(2**0.5)
# gauge and pitch for line 5-7 (they are equal):
s = g = (l56+l67)/(2**0.5)
# net area:
Ane = An = (l56+l67+l73)*1 - 2*hd*1 + (s*s/(4*g))*1       # 12.3.1
l56,l67,l73,s, Ane

(49.497, 106.066, 110, 110.000, 245.063)

In [20]:
tr1 = phiu*Ane*pFu * 1E-3      # 13.2 iii)
tr1

80.871

##### Bearing resistance  (1mm plate)

In [21]:
br1 = 3*phibr*6*1*bd*pFu * 1E-3
br1

120.701

In [22]:
# governing resistance is (the minimum of the above)
r = min(vr1,vr2,vr3,tr1,br1)
r

45.455

In [23]:
# thickness required, mm
Tf/r

17.600

The governing resistance is block shear (Path 1-3).  The required thickness is thus 17.6mm, so try a 20mm gusset plate.

### Try 20mm Gusset Plate

## 20mm Insert Plate (Slotted into HSS)

In [25]:
Dsg = 'HS127x127x6.4'
hss = sst.section(Dsg)
hssT,hssAg = hss.props('T,A')
hssT,hssAg

(6.350, 2960.000)

In [34]:
pW = 35 + 75 + 35   # width of plate based on bolt spacing and edge distance
pT = 20             # thickness of plate based on gusset thickness

##### Gross Area Yield

In [35]:
Ag = hssAg
Tr = phi*Ag*pFy * 1E-3
Tr, Tr >= Tf

(799.200, False)

Thats close enough.  OK

##### Net Section Fracture

In [37]:
Ane = pW*pT - 2*hd*pT
Tr = phiu*Ane*pFu * 1E-3
Tr, Tr >= Tf

(640.200, False)

No good - make the plate wider from:

$$
\phi_u(w t - 2\times24t)F_u \ge T_f
$$

$$
w t \ge \frac{T_f}{\phi_u F_u} + 48 t
$$

$$
w \ge \frac{T_f}{\phi_u t F_u} + 48
$$

In [39]:
Tf/(phiu * pT * pFu*1E-3) + 48

169.212

### Try 20mm x 170mm Insert plate

This leads $(170-75)/2 \approx 50$mm edge distances.  As the plate has to be so wide anyway (I'm relunctant to
make it thicker because of the extra cross-section removed from the HSS), perhaps we should try the 3x2 bolt arrangement and a slightly wider plate.

### Try 20mm x 220mm Insert Plate

![Dimensions](insert-3x2-dimensions.svg)

![Paths](insert-3x2-paths.svg)